In [1]:
%load_ext autoreload
%autoreload 2

Activate VENV


In [10]:
%%bash

pip install pyspark polars pandas setuptools

  Using cached setuptools-79.0.0-py3-none-any.whl.metadata (6.5 kB)
Using cached setuptools-79.0.0-py3-none-any.whl (1.3 MB)


In [11]:
%%bash

pip list

Package                 Version
----------------------- -----------
asttokens               3.0.0
comm                    0.2.2
debugpy                 1.8.14
decorator               5.2.1
executing               2.2.0
ipykernel               6.29.5
ipython                 9.1.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
jupyter_client          8.6.3
jupyter_core            5.7.2
matplotlib-inline       0.1.7
nest-asyncio            1.6.0
numpy                   2.2.5
packaging               25.0
pandas                  2.2.3
parso                   0.8.4
pexpect                 4.9.0
pip                     25.0.1
platformdirs            4.3.7
polars                  1.27.1
prompt_toolkit          3.0.51
psutil                  7.0.0
ptyprocess              0.7.0
pure_eval               0.2.3
py4j                    0.10.9.7
Pygments                2.19.1
pyspark                 3.5.5
python-dateutil         2.9.0.post0
pytz                    2025.2
pyzmq           

In [4]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
.appName("Polars experiment") \
.getOrCreate()
spark

your 131072x1 screen size is bogus. expect trouble
25/04/22 22:38:52 WARN Utils: Your hostname, DESKTOP-73AGGUC resolves to a loopback address: 127.0.1.1; using 172.19.139.222 instead (on interface eth0)
25/04/22 22:38:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/22 22:38:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
path = "data/user-demographics.csv"

df_base = spark.read.option("delimiter", ",").option("header", True).csv(path)
df_doubled = df_base.union(df_base)

In [14]:
df_base.show()

+-------------+--------------+-----+---------+----------------+-------------------+-------------------------+-----------------------+-----------+
|    Town/City|       Country|Users|New users|Engaged sessions|    Engagement rate|Engaged sessions per user|Average engagement time|Event count|
+-------------+--------------+-----+---------+----------------+-------------------+-------------------------+-----------------------+-----------+
|    Hyderabad|         India|75090|    71123|           81835| 0.5604676327973040|       1.0898255426821100|     132.49969370089200|     758313|
|    Bengaluru|         India|74325|    72162|           84798| 0.5364585310305560|       1.1409081735620600|      138.7289875546590|     776356|
|      Chennai|         India|54031|    51207|           55398| 0.5763720543099410|       1.0253002905739300|     120.58566378560500|     511249|
|       Mumbai|         India|48323|    45147|           43751| 0.4837143994339290|       0.9053866688740350|     103.302547

In [ ]:
df_base.count()
df_doubled.count()

10000

In [8]:
from datetime import datetime

In [33]:
start_time = datetime.now()
end_time = datetime.now()
print(f"Time elapsed for processing: {end_time-start_time}")


Time elapsed for processing: 0:00:00.000089


Experiment spark df to pandas df

In [ ]:
# Workaround for pyspark 3.5.x with python 3.12 while not supported yet
import setuptools
from distutils.version import LooseVersion

In [13]:
import pandas as pd

In [18]:
start_time = datetime.now()

pd_base = df_base.toPandas()

end_time = datetime.now()
print(f"Time elapsed for processing spark to pandas df: {end_time-start_time}")

Time elapsed for processing spark to pandas df: 0:00:00.368881


In [23]:
pd_base

,Town/City,Country,Users,New users,Engaged sessions,Engagement rate,Engaged sessions per user,Average engagement time,Event count
0,Hyderabad,India,75090,71123,81835,0.5604676327973040,1.0898255426821100,132.49969370089200,758313
1,Bengaluru,India,74325,72162,84798,0.5364585310305560,1.1409081735620600,138.7289875546590,776356
2,Chennai,India,54031,51207,55398,0.5763720543099410,1.0253002905739300,120.58566378560500,511249
3,Mumbai,India,48323,45147,43751,0.4837143994339290,0.9053866688740350,103.30254744117700,422074
4,Pune,India,44515,41678,45887,0.5314009102384450,1.0308210715489200,119.91375940694100,420101
...,...,...,...,...,...,...,...,...,...
4995,Chestermere,Canada,12,11,8,0.6153846153846150,0.6666666666666670,16.083333333333300,61
4996,Chilpancingo,Mexico,12,12,2,0.13333333333333300,0.16666666666666700,3.25,45
4997,Chokio,United States,12,9,5,0.3333333333333330,0.4166666666666670,18.916666666666700,50
4998,Chorley,United Kingdom,12,12,11,0.7857142857142860,0.9166666666666670,239.5,57
